In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes


!pip install sacrebleu google-cloud-translate

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hy9d2mno/unsloth_d97bba9e14074f8cb300be0f900843bf
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hy9d2mno/unsloth_d97bba9e14074f8cb300be0f900843bf
  Resolved https://github.com/unslothai/unsloth.git to commit 229e2ecc67756f36316dfcbea42396f59eef44e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

!pip install sacrebleu google-cloud-translate


from huggingface_hub import notebook_login
notebook_login()

In [ ]:


import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split


SYNTHETIC_FILE = "/content/drive/MyDrive/synthetic_data.csv"


SYSTEM_PROMPT = "You are a highly skilled translator. Your task is to accurately translate English text to Tamil, preserving the original meaning and tone."

def format_dataset(row):
    """
    Formats each row of data into the Gemma chat template. Gemma does not support 'system' role,
    so merge it into the 'user' message. Use 'model' role for assistant responses.
    """
    english_text = row['english']
    tamil_text = row['tamil']

    user_content = f"{SYSTEM_PROMPT}\n\n{english_text}"
    return {
        "messages": [
            {"role": "user", "content": user_content},
            {"role": "model", "content": tamil_text}
        ]
    }

print("--- Preparing Dataset ---")

try:
    df = pd.read_csv(SYNTHETIC_FILE)
    print(f"Loaded {len(df)} rows from {SYNTHETIC_FILE}")
except FileNotFoundError:
    print(f"ERROR: Make sure you have uploaded '{SYNTHETIC_FILE}' to the Colab session.")
    raise

# Clean data and apply the formatting
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
formatted_data = df.apply(format_dataset, axis=1).tolist()

# Split into training and evaluation sets
train_data, eval_data = train_test_split(formatted_data, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset objects
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

print(f"\nCreated {len(train_dataset)} training examples and {len(eval_dataset)} evaluation examples.")
print("\nVerifying the first training example:")
print(train_dataset[0]['messages'])
print("\n✅ Data preparation complete.")

--- Preparing Dataset ---
Loaded 2044 rows from /content/drive/MyDrive/synthetic_data.csv

Created 1836 training examples and 204 evaluation examples.

Verifying the first training example:
[{'content': 'You are a highly skilled translator. Your task is to accurately translate English text to Tamil, preserving the original meaning and tone.\n\nMy grandmother makes the best sweets.', 'role': 'user'}, {'content': ' என் பாட்டி சிறந்த இனிப்புகளைச் செய்வார்.', 'role': 'model'}]

✅ Data preparation complete.


In [ ]:


from unsloth import FastLanguageModel
import torch
from transformers import TrainingArguments
from trl import SFTTrainer

# --- Configuration ---
MODEL_NAME = "unsloth/gemma-3-4b-it-bnb-4bit"  # Gemma 3 4B instruct, quantized for efficiency
MAX_SEQ_LENGTH = 8192

# 1. Load the model using Unsloth's FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

tokenizer_inner = tokenizer.tokenizer


def format_example(example):

    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

print("--- Applying chat template to datasets ---")
train_dataset = train_dataset.map(format_example)
eval_dataset = eval_dataset.map(format_example)
print("\nVerifying formatted text (first example):")
print(train_dataset[0]['text'])
print("\n✅ Datasets formatted.")


model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer_inner,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=2,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        save_strategy="epoch",
        eval_strategy="epoch",
    ),
)


print("--- Starting Fine-Tuning ---")
trainer_stats = trainer.train()

print("\n✅ Fine-tuning complete!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.7: Fast Gemma3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Epoch,Training Loss,Validation Loss
1,0.410800,0.563631
2,0.499900,0.555322


Unsloth: Will smartly offload gradients to save VRAM!


Unsloth: Not an error, but Gemma3ForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient



✅ Fine-tuning complete!


In [ ]:
from unsloth import FastLanguageModel
import torch


SYSTEM_PROMPT = """
You are a highly skilled translator specializing in English to Tamil translation. Your task is to:
1. Accurately translate the provided English sentence into Tamil, preserving the original meaning, tone, and context.
2. Provide only the translated Tamil sentence as output, without any additional text, explanations, or repeated translations.
3. Ensure the translation is grammatically correct, natural, and culturally appropriate for Tamil speakers.
4. Do not include any English text or extra phrases in the output.
"""


def generate_translation(model, tokenizer, english_text, system_prompt):
    """
    Generate Tamil translation for a given English sentence using the fine-tuned model.
    """

    user_content = f"{system_prompt}\n\nEnglish: {english_text}"
    messages = [{"role": "user", "content": user_content}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        use_cache=True,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    generated_ids = outputs[0][len(inputs["input_ids"][0]):]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()


model = FastLanguageModel.for_inference(model)  # Optimize for faster generation
model.eval()  # Set model to evaluation mode
model.to("cuda")  # Ensure model is on GPU

# --- Interactive Translation Loop ---
print("--- Interactive English to Tamil Translator ---")
print("Enter an English sentence to translate into Tamil (or type 'quit' to exit).")
while True:
    english_input = input("English sentence: ")
    if english_input.lower() == 'quit':
        break
    if not english_input.strip():
        print("Please enter a valid sentence.")
        continue
    translation = generate_translation(model, tokenizer, english_input, SYSTEM_PROMPT)
    print(f"Tamil translation: {translation}\n")
print("✅ Translator closed.")

--- Interactive English to Tamil Translator ---
Enter an English sentence to translate into Tamil (or type 'quit' to exit).
English sentence: I want to visit a historical place in Tamil Nadu.
Tamil translation: நான் தமிழ்நாட்டில் ஒரு வரலாற்று இடத்தைப் பார்வையிட விரும்புகிறேன்.

English sentence: The old lighthouse keeper, whose face was weathered by a lifetime of sea salt and relentless wind, watched with a quiet, knowing gaze as the tempestuous storm, which had been brewing on the horizon for days, finally unleashed its full fury upon the jagged coastline, even though he knew his sturdy, stone-built tower would withstand the punishing waves and the furious, driving rain.
Tamil translation: கடல் உப்பு மற்றும் தொடர்ச்சியான காற்று மற்றும் ஒரு வாழ்நாள் முழுவதும் வழுவழுப்பான முகத்துடன், ஒரு அமைதியான, அறிவார்ந்த பார்வையைப் பயன்படுத்தி, பல நாட்களாக அடிவாரத்தில் கொந்தளித்த ஒரு கொந்தளிப்பான புயல் இறுதியாக வழுவழுப்பான கடற்கரையில் அதன் முழு கொந்தளிப்பையும் unleashed செய்தபோது, ​​அவர் தனது உறுதிய

In [ ]:


import os
from google.colab import auth
from google.cloud import translate_v2 as translate
from unsloth import FastLanguageModel

# --- Google Translate Setup ---
auth.authenticate_user()
# IMPORTANT: Replace these with your GCP Project ID and the API Key you created
GCP_PROJECT_ID = ""
GOOGLE_API_KEY = ""
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
translate_client = translate.Client()

# --- Base Gemma Model (Un-tuned) ---
print("Loading base Gemma 3 model...")
base_gemma_model, base_gemma_tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME, max_seq_length=MAX_SEQ_LENGTH, dtype=None, load_in_4bit=True,
)

# --- Llama 3 Model ---
print("\nLoading Llama 3 model...")
llama_model, llama_tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit", max_seq_length=MAX_SEQ_LENGTH, dtype=None, load_in_4bit=True,
)

print("\n✅ All baseline models are ready.")

Loading base Gemma 3 model...
==((====))==  Unsloth 2025.9.7: Fast Gemma3 patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.

Loading Llama 3 model...
==((====))==  Unsloth 2025.9.7: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore 

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]


✅ All baseline models are ready.


In [ ]:
!pip install --upgrade sacrebleu

In [ ]:
import sacrebleu
import pandas as pd
from tqdm import tqdm

def generate_translation(model, tokenizer, english_text, system_prompt):
    """
    Generate Tamil translation for a given English sentence using the fine-tuned model.
    Handles both Gemma and Llama chat templates.
    """
    if "gemma" in model.config.model_type:
        user_content = f"{system_prompt}\n\nEnglish: {english_text}"
        messages = [{"role": "user", "content": user_content}]
    else:  # Assume Llama 3
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": english_text},
        ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        use_cache=True,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    generated_ids = outputs[0][len(inputs["input_ids"][0]):]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

# Test the helper function with a sample input
try:
    test_text = "Hello, everyone!"
    test_prompt = "You are a highly skilled translator. Translate the following English text to Tamil accurately."
    test_translation = generate_translation(model, tokenizer, test_text, test_prompt)
    print(f"Test translation: {test_translation}")
except Exception as e:
    print(f"Error in generate_translation: {e}")

Test translation: வணக்கம், எல்லோருக்கும்!


In [ ]:
references = []
finetuned_gemma_translations = []
base_gemma_translations = []
llama3_translations = []

print("\n--- Starting evaluation loop ---")
for item in tqdm(eval_dataset, desc="Evaluating"):
    try:
        # Extract English and reference Tamil texts
        user_content = item['messages'][0]['content']
        english_text_start_index = user_content.find("\n\n") + 2
        eng_text = user_content[english_text_start_index:].strip()
        ref_text = item['messages'][1]['content']

        # Store reference
        references.append(ref_text)

        # Generate translations
        finetuned_trans = generate_translation(model, tokenizer, eng_text, SYSTEM_PROMPT)
        base_gemma_trans = generate_translation(base_gemma_model, base_gemma_tokenizer, eng_text, SYSTEM_PROMPT)
        llama3_trans = generate_translation(llama_model, llama_tokenizer, eng_text, SYSTEM_PROMPT)

        # Store translations
        finetuned_gemma_translations.append(finetuned_trans)
        base_gemma_translations.append(base_gemma_trans)
        llama3_translations.append(llama3_trans)

        # Print sample for verification
        if len(references) <= 3:  # Print first 3 for debugging
            print(f"\nSample {len(references)}:")
            print(f"English: {eng_text}")
            print(f"Reference: {ref_text}")
            print(f"Fine-tuned Gemma: {finetuned_trans}")
            print(f"Base Gemma: {base_gemma_trans}")
            print(f"Llama 3: {llama3_trans}")

    except Exception as e:
        print(f"Error processing item {item}: {e}")
        references.append("")
        finetuned_gemma_translations.append("")
        base_gemma_translations.append("")
        llama3_translations.append("")

print(f"\nGenerated {len(references)} translations.")
print(f"Sample reference: {references[0] if references else 'None'}")


--- Starting evaluation loop ---


Evaluating:   0%|          | 1/204 [00:20<1:08:30, 20.25s/it]


Sample 1:
English: The project will have been completed.
Reference: திட்டம் முடிக்கப்பட்டிருக்கும்.
Fine-tuned Gemma: திட்டம் முடிந்துவிட்டது.
Base Gemma: திட்டம் முடிந்துவிட்டது.
Llama 3: திட்டம் முடிக்கப்பட்டுவிடும்.


Evaluating:   1%|          | 2/204 [00:32<51:45, 15.37s/it]  


Sample 2:
English: You should practice a little every day.
Reference:  நீங்கள் ஒவ்வொரு நாளும் கொஞ்சம் பயிற்சி செய்ய வேண்டும்.
Fine-tuned Gemma: நீங்கள் ஒவ்வொரு நாளும் ஒரு சிறிய அளவாவது பயிற்சி செய்ய வேண்டும்.
Base Gemma: ஒவ்வொரு நாளும் கொஞ்சம் பயிற்சி செய்யுங்கள்.
Llama 3: நாட்டில் ஒரு நாள் பயிற்சி செய்யுங்கள்.


Evaluating:   1%|▏         | 3/204 [00:41<42:16, 12.62s/it]


Sample 3:
English: Festivals in the village are very traditional.
Reference:  கிராமத்தில் பண்டிகைகள் மிகவும் பாரம்பரியமாக இருக்கும்.
Fine-tuned Gemma: கிராமத்தில் உள்ள திருவிழாக்கள் மிகவும் பாரம்பரியமானவை.
Base Gemma: கிராமத்து பண்டிகங்கள் மிகவும் பாரம்பரியமானவை.
Llama 3: கிராமத்தில் விழாக்கள் வரலாற்றுப்படி ஆகும்.


Evaluating: 100%|██████████| 204/204 [31:34<00:00,  9.29s/it]


Generated 204 translations.
Sample reference: திட்டம் முடிக்கப்பட்டிருக்கும்.


In [ ]:
# Calculating BLEU and chrF++ scores
print("\n--- Calculating BLEU and chrF++ scores ---")
try:
    chrf = sacrebleu.metrics.CHRF(word_order=2)  # chrF++ metric

    # Fine-tuned Model Scores
    bleu_finetuned = sacrebleu.corpus_bleu(finetuned_gemma_translations, [references])
    chrf_finetuned = chrf.corpus_score(finetuned_gemma_translations, [references])

    # Base Gemma Scores
    bleu_base_gemma = sacrebleu.corpus_bleu(base_gemma_translations, [references])
    chrf_base_gemma = chrf.corpus_score(base_gemma_translations, [references])

    # Llama 3 Scores
    bleu_llama3 = sacrebleu.corpus_bleu(llama3_translations, [references])
    chrf_llama3 = chrf.corpus_score(llama3_translations, [references])

    # Display Results
    results = {
        "Model": ["Your Fine-tuned Gemma", "Base Gemma (Untuned)", "Llama 3 8B Instruct"],
        "BLEU Score": [f"{bleu_finetuned.score:.2f}", f"{bleu_base_gemma.score:.2f}", f"{bleu_llama3.score:.2f}"],
        "chrF++ Score": [f"{chrf_finetuned.score:.2f}", f"{chrf_base_gemma.score:.2f}", f"{chrf_llama3.score:.2f}"]
    }
    results_df = pd.DataFrame(results)
    print("\n--- Evaluation Results ---")
    print(results_df.to_string(index=False))

except Exception as e:
    print(f"Error calculating metrics: {e}")


--- Calculating BLEU and chrF++ scores ---

--- Evaluation Results ---
                Model BLEU Score chrF++ Score
Your Fine-tuned Gemma      36.12        65.25
 Base Gemma (Untuned)      28.84        59.95
  Llama 3 8B Instruct       2.94        21.35


In [ ]:
import torch

# --- 1. Save LoRA adapters ---
try:
    model.save_pretrained("gemma_tamil_translator_lora")
    tokenizer.save_pretrained("gemma_tamil_translator_lora")
    print("✅ LoRA adapters saved to 'gemma_tamil_translator_lora'")
except Exception as e:
    print(f"⚠️ Error saving LoRA adapters: {str(e)}")

# --- 2. Merge adapters and save in full PyTorch format ---
try:
    merged_model = model.merge_and_unload()
    merged_model.save_pretrained("gemma_tamil_translator_full_pytorch", safe_serialization=True)
    tokenizer.save_pretrained("gemma_tamil_translator_full_pytorch")
    print("\n✅ Full PyTorch model saved to 'gemma_tamil_translator_full_pytorch' (in .safetensors format)")
except Exception as e:
    print(f"⚠️ Error merging and saving PyTorch model: {str(e)}")

# --- 2a. Save merged model as a single .pt file ---
try:
    torch.save(merged_model.state_dict(), "gemma_tamil_translator.pt")
    print("\n✅ Merged model weights saved as single 'gemma_tamil_translator.pt' file")
except Exception as e:
    print(f"⚠️ Error saving .pt file: {str(e)}")

✅ LoRA adapters saved to 'gemma_tamil_translator_lora'

✅ Full PyTorch model saved to 'gemma_tamil_translator_full_pytorch' (in .safetensors format)

✅ Merged model weights saved as single 'gemma_tamil_translator.pt' file


In [ ]:
import os
import torch
from unsloth import FastLanguageModel
from huggingface_hub import HfApi, upload_folder
from transformers import AutoModelForCausalLM, AutoTokenizer

# --- Install required packages ---
try:
    !pip install huggingface_hub
    print("✅ Installed huggingface_hub package")
except Exception as e:
    print(f"⚠️ Error installing huggingface_hub: {str(e)}")

# --- Get Hugging Face token ---
try:
    # Assuming HF_TOKEN is set in Colab secrets
    from google.colab import userdata
    HF_TOKEN = userdata.get("HF_TOKEN")
    if not HF_TOKEN:
        raise ValueError("HF_TOKEN not found in Colab secrets.")
except Exception as e:
    print(f"⚠️ Error accessing HF_TOKEN: {str(e)}")
    print("Set HF_TOKEN in Colab secrets (key icon > Add new secret > Name: HF_TOKEN, Value: your_token)")
    raise

# --- Define repository name ---
REPO_ID = "arsath-sm/gemma3-tamil-translator"
LOCAL_MODEL_PATH = "gemma_tamil_translator_full_pytorch" # Path to the saved full model

# --- Push saved model files from disk to Hugging Face Hub ---
print(f"\n--- Pushing model files from '{LOCAL_MODEL_PATH}' to Hugging Face Hub at '{REPO_ID}' ---")
try:
    api = HfApi()

    # Create the repository if it doesn't exist (optional, but good practice)
    # You might need to handle potential errors if the repo already exists
    try:
        api.create_repo(repo_id=REPO_ID, token=HF_TOKEN, repo_type="model")
        print(f"✅ Created repository: {REPO_ID}")
    except Exception as e:
        print(f"ℹ️ Repository {REPO_ID} might already exist or there was an error creating it: {e}")
        # Continue assuming the repo exists

    # Upload the entire folder containing the model files
    upload_folder(
        folder_path=LOCAL_MODEL_PATH,
        repo_id=REPO_ID,
        repo_type="model",
        token=HF_TOKEN,
        commit_message=f"Upload {REPO_ID} model files"
    )

    print(f"\n✅ Model files successfully pushed to Hugging Face Hub at {REPO_ID}")
    print("You can now use it via the free Inference API or load it with its name.")
except Exception as e:
    print(f"⚠️ Error pushing to Hugging Face Hub: {str(e)}")
    print("Check HF_TOKEN, REPO_ID, or network connectivity. Ensure the repository doesn't already exist.")

✅ Installed huggingface_hub package

--- Pushing model files from 'gemma_tamil_translator_full_pytorch' to Hugging Face Hub at 'arsath-sm/gemma3-tamil-translator' ---
✅ Created repository: arsath-sm/gemma3-tamil-translator


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...l_pytorch/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...ll_pytorch/tokenizer.json:  50%|#####     | 16.7MB / 33.4MB            

  ...pytorch/model.safetensors:   0%|          | 5.37MB / 3.23GB            


✅ Model files successfully pushed to Hugging Face Hub at arsath-sm/gemma3-tamil-translator
You can now use it via the free Inference API or load it with its name.


In [ ]:
import os
import zipfile
from google.colab import files
from datetime import datetime

# --- Configuration ---
# List of files and directories to include in the zip
FILES_TO_ZIP = [
    "gemma_tamil_translator_lora",           # LoRA adapters directory
    "gemma_tamil_translator_full_pytorch",   # Full PyTorch model directory
    "gemma_tamil_translator.pt",             # Single .pt file
    "unsloth_compiled_cache",
    "wandb"
    "outputs"                               # Training outputs (checkpoints, logs)
]

# Name the zip file with a timestamp for uniqueness
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
ZIP_FILE = f"gemma_tamil_translator_files_{timestamp}.zip"

# --- Create the zip file ---
print(f"\n--- Creating zip file: {ZIP_FILE} ---")
with zipfile.ZipFile(ZIP_FILE, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for item in FILES_TO_ZIP:
        if os.path.exists(item):
            if os.path.isfile(item):
                # If it's a file, add it directly
                print(f"Adding file: {item}")
                zipf.write(item, os.path.join("gemma_tamil_translator_files", item))
            elif os.path.isdir(item):
                # If it's a directory, walk through and add all files
                print(f"Adding directory: {item}")
                for root, _, files_in_dir in os.walk(item):
                    for file_name in files_in_dir:
                        file_path = os.path.join(root, file_name)
                        # Store with relative path inside zip
                        arcname = os.path.join("gemma_tamil_translator_files", root, file_name)
                        print(f"  Adding: {file_path}")
                        zipf.write(file_path, arcname)
        else:
            print(f"⚠️ Item not found: {item}")

print(f"\n✅ Zip file created: {ZIP_FILE}")

# --- Download the zip file ---
print("\n--- Initiating download ---")
files.download(ZIP_FILE)
print("✅ Download initiated. Check your browser's download folder.")


--- Creating zip file: gemma_tamil_translator_files_20250924_091234.zip ---
Adding directory: gemma_tamil_translator_lora
  Adding: gemma_tamil_translator_lora/processor_config.json
  Adding: gemma_tamil_translator_lora/added_tokens.json
  Adding: gemma_tamil_translator_lora/tokenizer.json
  Adding: gemma_tamil_translator_lora/tokenizer_config.json
  Adding: gemma_tamil_translator_lora/chat_template.jinja
  Adding: gemma_tamil_translator_lora/special_tokens_map.json
  Adding: gemma_tamil_translator_lora/tokenizer.model
  Adding: gemma_tamil_translator_lora/README.md
  Adding: gemma_tamil_translator_lora/preprocessor_config.json
  Adding: gemma_tamil_translator_lora/adapter_config.json
  Adding: gemma_tamil_translator_lora/adapter_model.safetensors
Adding directory: gemma_tamil_translator_full_pytorch
  Adding: gemma_tamil_translator_full_pytorch/processor_config.json
  Adding: gemma_tamil_translator_full_pytorch/added_tokens.json
  Adding: gemma_tamil_translator_full_pytorch/tokenizer

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download initiated. Check your browser's download folder.
